<a href="https://colab.research.google.com/github/prathibaleri20/AI-Based-Cyber-Security-Threats-Prediction-AI-Agent/blob/main/SQL_TASK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd

# Create in-memory database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

Create a table employees with columns: id, name, department, salary, join_date. Insert 5 sample rows.


In [2]:
cursor.execute('''
CREATE TABLE employees (
    id INTEGER,
    name TEXT,
    department TEXT,
    salary INTEGER,
    join_date TEXT
)
''')

cursor.executemany('''
INSERT INTO employees VALUES (?, ?, ?, ?, ?)
''', [
    (1, 'Asha', 'HR', 50000, '2022-01-10'),
    (2, 'Rahul', 'IT', 70000, '2021-11-05'),
    (3, 'Priya', 'Finance', 65000, '2023-03-15'),
    (4, 'Kiran', 'IT', 72000, '2020-07-20'),
    (5, 'Meena', 'HR', 52000, '2022-06-30')
])

Prompt: Show all employees from the IT department earning more than ₹70,000.

In [4]:
query = "SELECT * FROM employees WHERE department = 'IT' AND salary > 70000"
it_employees = pd.read_sql_query(query, conn)
display(it_employees)

,id,name,department,salary,join_date
0,4,Kiran,IT,72000,2020-07-20


Prompt:Show average salary per department using group by

In [5]:
query = "SELECT department, AVG(salary) AS average_salary FROM employees GROUP BY department"
average_salary_per_department = pd.read_sql_query(query, conn)
display(average_salary_per_department)

,department,average_salary
0,Finance,65000.0
1,HR,51000.0
2,IT,71000.0


Prompt: List employees by highest salary using order by


In [7]:
query = "SELECT * FROM employees ORDER BY salary DESC"
employees_by_salary = pd.read_sql_query(query, conn)
display(employees_by_salary)

,id,name,department,salary,join_date
0,4,Kiran,IT,72000,2020-07-20
1,2,Rahul,IT,70000,2021-11-05
2,3,Priya,Finance,65000,2023-03-15
3,5,Meena,HR,52000,2022-06-30
4,1,Asha,HR,50000,2022-01-10


Prompt:Show employees who joined after Jan 1, 2022.

In [8]:
df = pd.read_sql_query('''
SELECT * FROM employees
WHERE join_date > '2022-01-01'
''', conn)
df

,id,name,department,salary,join_date
0,1,Asha,HR,50000,2022-01-10
1,3,Priya,Finance,65000,2023-03-15
2,5,Meena,HR,52000,2022-06-30


Prompt: Create a second table projects and join with employees to show who is working on what.


In [9]:
cursor.execute('''
CREATE TABLE projects (
    emp_id INTEGER,
    project_name TEXT
)
''')

cursor.executemany('''
INSERT INTO projects VALUES (?, ?)
''', [
    (1, 'Onboarding'),
    (2, 'AI Chatbot'),
    (3, 'Budget Tracker'),
    (4, 'Cyber Threat Agent')
])

df = pd.read_sql_query('''
SELECT e.name, e.department, p.project_name
FROM employees e
JOIN projects p ON e.id = p.emp_id
''', conn)
df

,name,department,project_name
0,Asha,HR,Onboarding
1,Rahul,IT,AI Chatbot
2,Priya,Finance,Budget Tracker
3,Kiran,IT,Cyber Threat Agent


Prompt: show inner join right join left join self join with example

### Inner Join

An INNER JOIN returns only the rows where there is a match in both tables.

In [10]:
query = """
SELECT e.name, e.department, p.project_name
FROM employees e
INNER JOIN projects p ON e.id = p.emp_id
"""
inner_join_df = pd.read_sql_query(query, conn)
display(inner_join_df)

,name,department,project_name
0,Asha,HR,Onboarding
1,Rahul,IT,AI Chatbot
2,Priya,Finance,Budget Tracker
3,Kiran,IT,Cyber Threat Agent


### Left Join

A LEFT JOIN returns all rows from the left table, and the matched rows from the right table. If there is no match in the right table, the result is NULL on the right side.

In [11]:
query = """
SELECT e.name, e.department, p.project_name
FROM employees e
LEFT JOIN projects p ON e.id = p.emp_id
"""
left_join_df = pd.read_sql_query(query, conn)
display(left_join_df)

,name,department,project_name
0,Asha,HR,Onboarding
1,Rahul,IT,AI Chatbot
2,Priya,Finance,Budget Tracker
3,Kiran,IT,Cyber Threat Agent
4,Meena,HR,None


### Right Join

A RIGHT JOIN returns all rows from the right table, and the matched rows from the left table. If there is no match in the left table, the result is NULL on the left side.

*Note: SQLite does not directly support RIGHT JOIN. We can achieve a similar result by switching the table order and using a LEFT JOIN.*

In [12]:
query = """
SELECT e.name, e.department, p.project_name
FROM projects p
LEFT JOIN employees e ON p.emp_id = e.id
"""
right_join_df = pd.read_sql_query(query, conn)
display(right_join_df)

,name,department,project_name
0,Asha,HR,Onboarding
1,Rahul,IT,AI Chatbot
2,Priya,Finance,Budget Tracker
3,Kiran,IT,Cyber Threat Agent


### Self Join

A SELF JOIN is a regular join used to join a table to itself. It is used to compare rows within the same table.

In [13]:
# To demonstrate a self join, let's assume we want to find pairs of employees in the same department
query = """
SELECT a.name AS employee1, b.name AS employee2, a.department
FROM employees a, employees b
WHERE a.department = b.department AND a.id <> b.id
"""
self_join_df = pd.read_sql_query(query, conn)
display(self_join_df)

,employee1,employee2,department
0,Asha,Meena,HR
1,Rahul,Kiran,IT
2,Kiran,Rahul,IT
3,Meena,Asha,HR


Prompt: Show each employee’s salary and compare it with the previous and next employee.


In [14]:
cursor.execute('''
CREATE TABLE salaries (
    id INTEGER,
    name TEXT,
    salary INTEGER
)
''')

cursor.executemany('''
INSERT INTO salaries VALUES (?, ?, ?)
''', [
    (1, 'Asha', 50000),
    (2, 'Rahul', 70000),
    (3, 'Priya', 65000),
    (4, 'Kiran', 72000),
    (5, 'Meena', 52000)
])

df = pd.read_sql_query('''
SELECT
    id,
    name,
    salary,
    LAG(salary) OVER (ORDER BY id) AS prev_salary,
    LEAD(salary) OVER (ORDER BY id) AS next_salary
FROM salaries
''', conn)
df

,id,name,salary,prev_salary,next_salary
0,1,Asha,50000,NaN,70000.0
1,2,Rahul,70000,50000.0,65000.0
2,3,Priya,65000,70000.0,72000.0
3,4,Kiran,72000,65000.0,52000.0
4,5,Meena,52000,72000.0,NaN


Prompt: Show names that appear more than once.(Find the duplicate values)


In [20]:
query = "SELECT name, COUNT(*) FROM employees GROUP BY name HAVING COUNT(*) > 1"
duplicate_names_df = pd.read_sql_query(query, conn)
display(duplicate_names_df)

,name,COUNT(*)


Prompt: Convert department-wise monthly expenses into columns.


In [24]:
cursor.execute('''
CREATE TABLE monthly_expenses (
    month TEXT,
    department TEXT,
    expense INTEGER
)
''')

cursor.executemany('''
INSERT INTO monthly_expenses VALUES (?, ?, ?)
''', [
    ('Jan', 'HR', 10000),
    ('Jan', 'IT', 25000),
    ('Jan', 'Finance', 15000),
    ('Feb', 'HR', 12000),
    ('Feb', 'IT', 27000),
    ('Feb', 'Finance', 16000),
    ('Mar', 'HR', 11000),
    ('Mar', 'IT', 26000),
    ('Mar', 'Finance', 15500)
])

query = """
SELECT
    month,
    SUM(CASE WHEN department = 'HR' THEN expense ELSE 0 END) AS HR_Expense,
    SUM(CASE WHEN department = 'IT' THEN expense ELSE 0 END) AS IT_Expense,
    SUM(CASE WHEN department = 'Finance' THEN expense ELSE 0 END) AS Finance_Expense
FROM monthly_expenses
GROUP BY month
ORDER BY month;
"""

monthly_expenses_pivot_df = pd.read_sql_query(query, conn)
display(monthly_expenses_pivot_df)

,month,HR_Expense,IT_Expense,Finance_Expense
0,Feb,12000,27000,16000
1,Jan,10000,25000,15000
2,Mar,11000,26000,15500


Prompt:Unpivot it

In [26]:
df_unpivot = monthly_expenses_pivot_df.melt(id_vars=['month'], var_name='department', value_name='expense')
display(df_unpivot)

,month,department,expense
0,Feb,HR_Expense,12000
1,Jan,HR_Expense,10000
2,Mar,HR_Expense,11000
3,Feb,IT_Expense,27000
4,Jan,IT_Expense,25000
5,Mar,IT_Expense,26000
6,Feb,Finance_Expense,16000
7,Jan,Finance_Expense,15000
8,Mar,Finance_Expense,15500


Prompt: Combine first and last names into full name
by doing concatenation

In [27]:
cursor.execute('''
CREATE TABLE people (
    first_name TEXT,
    last_name TEXT
)
''')

cursor.executemany('''
INSERT INTO people VALUES (?, ?)
''', [
    ('Vivek', 'Sharma'),
    ('Asha', 'Rao'),
    ('Rahul', 'Mehta')
])

df = pd.read_sql_query('''
SELECT first_name || ' ' || last_name AS full_name
FROM people
''', conn)
df

,full_name
0,Vivek Sharma
1,Asha Rao
2,Rahul Mehta
